# DPH algorithm implementation

Dominance Principle was originally proposed for Multi Demand Knapsack in

> "A new polynomial time algorithm for 0–1 multiple knapsack problem based
        on dominant principles", 
        
by Balachandar and Kannan.
        
Dominance Principle was adapted to MDMKP in

> "A new heuristic approach for Knapsack/Covering Problem",

also by Balachandar and Kannan. This method is what this notebook implements. 

---

https://hub.gke.mybinder.org/user/dibillilia-jbm-amdxbrwc/notebooks/DPH.ipynb

---

First, we must include my code for metaheuristics for the MDMKP. This contains things like the problem definition and functions to score solutions, as well as the metaheuristics. 

In [3]:
include("compose.jl");

Now we are ready to implement the Dominance Principle Heuristic. We will do this step by step, so we can observe how each step changes the data. The first step involves sorting the problem demand constraint columns by totals of the columns, and going through this list, adding the sorted column's respective items into the knapsack, as long as at least one demand constraint is violated. 

In [4]:
"""Returns a bitlist of all zeros with the same length as the amount of problem dimensions. """
function generate_initial_bitarray(problem::Problem)
    initial_bitlist::BitArray = falses(length(problem.objective))
    
    demand_columns = [coeffs for (coeffs, bound) in problem.lower_bounds] #get list of demand constraint coefficients
    #convert list into matrix:
    demand_columns = permutedims(reshape(hcat(demand_columns...), (length(demand_columns[1]), length(demand_columns))))
    
    demand_bounds = [bound for (coeffs, bound) in problem.lower_bounds]
    demand_totals = [sum(demand_columns[:,i]) for i in 1:size(demand_columns)[2]]

    #we need to sort column_totals
    zipped::Vector{Tuple{Int,Int}} = []
    index = 0
    for total in demand_totals
        index += 1
        push!(zipped, tuple(total, index))
    end
    sort!(zipped, by=x->x[1], rev=true)
    
    #we can now loop over the columns in descending order
    for (column_total, column_index) in zipped
        
        #enable the bit
        initial_bitlist[column_index] = 1
        
        #decrease bounds by value of respective coefficients that were just enabled
        column_coefficients = demand_columns[:,column_index]
        for i in 1:length(column_coefficients)
            demand_bounds[i] -= column_coefficients[i]
        end
        
        #check that we should continue
        cont = false
        for bound in demand_bounds
            if bound > 0
                cont = true
                break
            end
        end
        if !cont
            break
        end
    end
    
    #we now have our initial starting bitlist
    return initial_bitlist
end

generate_initial_bitarray

Let's test this function:

In [5]:
problem = parse_file("./benchmark_problems/mdmkp_ct1.txt")[3]
initial_solution = generate_initial_bitarray(problem)
join([a ? "0" : "1" for a in initial_solution], ",")

"0,1,1,0,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,1,1,1,1,0,0,0,1,0,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0"

In [6]:
#lets check that the demand constraints are satisfied
satisfies_demands(initial_solution, problem)

true

In [7]:
#are the dimension constraints?
satisfies_dimensions(initial_solution,problem)

true

In [8]:
#for how many problems will this heuristic generate a feasible solution?
total_valid = 0
for dataset in 1:9
    problems = parse_file("./benchmark_problems/mdmkp_ct$(dataset).txt")
    for problem in problems
        ini_sol = generate_initial_bitarray(problem)
        if is_valid(ini_sol, problem)
            total_valid += 1
        end
    end
end
total_valid

616

In [58]:
#now we must construct the dimension intercept matrix
function redundant_minification(problem::Problem, sol::BitArray)
    sol = deepcopy(sol)
    
    n = length(problem.objective) #number of dimensions, width of coeff matrix, length of bitlist
    m = length(problem.upper_bounds) #number of dimension constraints
    M = 2^62 #"M a large value"
    
    A = [coeffs for (coeffs, bound) in problem.upper_bounds]
    A = permutedims(reshape(hcat(A...), (length(A[1]), length(A))))

    B = [bound for (coeffs, bound) in problem.upper_bounds]
    
    #our objective function has some negative values
    #to disable a column, we set it to 0
    #negatives are less than 0, so we end up selecting disabled columns before we select negative columns
    #this is bad
    #to fix it, we make negative objective values positive, but very small
    c::Vector{Float64} = [obj > 0 ? obj : abs(obj)/1000 for obj in problem.objective]

    #loop while A contains positive value
    discovered_solutions::Population = []
    while max(A...) > 0
        
        #STEP 3:
        #construct intercept matrix
        #next we "Form the matrix D[i,j] for i=1,2,3...m and j =1,2,3,...n",
        #where m is the amount of constraints and n is number of dimensions
        D = Array{Float64,2}(undef,m,n) #d is an m by n matrix
        for i in 1:m
            for j in 1:n
                D[i,j] = A[i,j] > 0 ?
                    B[i]/A[i,j] :
                    0 #the paper says to set this to a large value, but that results in the same column
                        #being selected over and over
            end
        end

        
#         #STEP 4:
#         #if any column of D contains a value < 1, set the whole column of A to 0 and the respective 
#         #solution bit to 0
#         for column_index in 1:size(D)[2]
#             column = D[:,column_index]
#             if min(column...) < 1
#                 A[:,column_index] = zeros(size(A)[1])
#                 sol[column_index] = 0
#             end
#         end

        #STEP 5
        #take list of minimum of each column of D
        redundant_variables = [min(D[:,i]...) for i in 1:size(D)[2]]

        #STEP 6
        #get index of largest product of smallest value of D times respective objective function coefficient
        products::Vector{Float64} = redundant_variables .* c
        if max(products...) == 0.0 #check for exit condition
            break
        end
        l = argmax(redundant_variables .* c)

        #STEP 7
        #step 7 is really vague, but the basic idea seems to be to put l into the knapsack as long as feasibility
        #won't be violated
        #do we care about dimension feasibility, demand feasibility, or both?
        if !sol[l] #if the bit is 0
            sol[l] = true
            demand_feas = satisfies_demands(sol, problem)
            dimension_feas = satisfies_dimensions(sol, problem)
            if demand_feas && dimension_feas
                #we must update the bounds
                B .-= A[:,l]
            else
                sol[l] = false #put the bit back, its not valid
            end
        end
        #regardless if we put l into the knapsack or not, we must 0 A so that l will not be chosen again
        A[:,l] = zeros(size(A)[1])

    end
    return VND(sol, problem)
end

best_found_solution = redundant_minification(problem, initial_solution)
get_representation(best_found_solution)

"Solution([1,0,1,0,1,1,0,1,0,0,0,1,0,0,1,1,1,1,1,0,1,1,1,0,0,0,1,1,0,1,1,1,0,1,1,1,1,1,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,0,0,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1], 17681)"

In [11]:
#now we combine the initial population step and the redundancy minification step to get DPH
function DPH(problem::Problem)
    initial_bitarray = generate_initial_bitarray(problem)
    return redundant_minification(problem, initial_bitarray)
end

DPH (generic function with 1 method)

In [12]:
#let's define a simple genetic alg, for comparison
function simpleGA(problem::Problem)
    pop::Population = random_init(problem, 50, ls=VND, force_valid=false)
    P_meta_coord(pop, problem, VND, column_average_chances, use_random=true, random_n=1, time_limit=1)
    return get_best_solution(pop)
end

simpleGA (generic function with 1 method)

In [61]:
#lets loop over every problem in dataset 1, and compare the performance
for dataset in 7:8
    problems = parse_file("./benchmark_problems/mdmkp_ct$(dataset).txt")
    for problem in problems
        DPH_sol = DPH(problem)
        GA_sol = simpleGA(problem)
        println(GA_sol.score," ",DPH_sol.score)
    end
end

18267 15267
11468 -11929
-37 -8148
9615 6589
3775 -11929
-71 -8148
18014 14990
9944 -3713
-446 -11498
10626 9242
3828 -3713
-666 -11498
20013 16360
13634 -3129
-367 -9690
10488 9043
3657 -3129
-562 -9690
18747 17102
9534 -24646
-555 -2585
10331 6400
-2 -24646
-210 -2585
18396 13005
1001 -962
-703 -4869
9879 7160
1482 -962
-331 -4869
37390 32517
23701 -1705
-5 -8187
19382 17106
2515 -1705
503 -8187
40155 37928
2530 -579
-4 -10674
20619 19844
2589 -579
-77 -10674
36999 34121
1979 -984
-13 -4974
19650 18599
3141 -984
-219 -4974
39041 34573
2736 -889
-157 -9641
19329 17962
2717 -889
-365 -9641
40707 36058
26684 -6
-266 -39904
20018 18662
7981 2468
266 -39904
52557 50901
1073 -1577
-676 -6320
27978 27487
869 -1577
-1279 -6320
55859 49991
666 -682
-566 -6788
27099 25124
1365 -682
-346 -6788
60839 58138
2375 -2329
-25 -4064
28339 27067
1284 -2329
-84 -4064
57554 55455
2454 -344
-696 -14132
27147 26046
1527 -344
-817 -14132
54798 52939
-1 -1626
-504 -9618
27986 26490
1453 -1626
-484 -9618
5835